# Questões  <a name="questoes"></a>
1. [Carregando a base e verificando duplicações](#lendo)
2. [Missing Values I](#missing)
3. [Missing Values II](#missingII)
4. [Missing Values III](#missingIII)
5. [Missing Values IV](#missingIV)
6. [Missing Values V](#missingV)
7. [Missing Values VI](#missingVI)
8. [Categorização](#categorizacao)
9. [Renomear variaveis](#renomeando)

In [1]:
"""
Importando as bibliotecas
"""
import pandas as pd
import requests

# Base de nascidos vivos do DataSUS

O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

### 1. Carregando a base e verificando duplicações <a name="lendo"></a>

<div style="text-align: right"
     
[Voltar ao índice](#questoes)

Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você 
aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. Há linhas
duplicadas?
     

In [2]:
"""
Não há linhas duplicadas.
"""
df_sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(df_sinasc.shape)
df_sinasc.drop_duplicates().shape

(27028, 69)


(27028, 69)

### 2. Missing Values I <a name="missing"></a>

<div style="text-align: right"
     
[Voltar ao índice](#questoes)
     
Conte o número de valores missing por variável.

In [3]:
df_sinasc.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

### 3. Missing Values II <a name="missingII"></a>

<div style="text-align: right"
     
[Voltar ao índice](#questoes)

Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser 
interesantes.Então crie uma seleção dessa base somente com as colunas que interessam.
São elas: ['LOCNASC','IDADEMAE','ESTCIVMAE','ESCMAE','QTDFILVIVO','GESTACAO','GRAVIDEZ','CONSULTAS','APGAR5']. Refaça a contagem de valores missings.

In [4]:
df_sinasc_sel = df_sinasc[['LOCNASC','IDADEMAE','ESTCIVMAE','ESCMAE','QTDFILVIVO','GESTACAO','GRAVIDEZ','CONSULTAS','APGAR5']]
df_sinasc_sel.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

### 4. Missing Values III <a name="missingIII"></a>

<div style="text-align: right"
     
[Voltar ao índice](#questoes)
     
Apgar é uma nota que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à 
respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal.
Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de 
missings.

In [5]:
df_sinasc_sel.dropna(subset =['APGAR5'], inplace = True)
df_sinasc_sel.isna().sum()

C:\Users\Natanael_Quintiliano\AppData\Local\Temp\ipykernel_16824\760065710.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sinasc_sel.dropna(subset =['APGAR5'], inplace = True)


LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

### 5. Missing Values IV <a name="missingIV"></a>

<div style="text-align: right"
     
[Voltar ao índice](#questoes)
     
Observe que as variáveis ['ESTCIVMAE','CONSULTAS'] possuem o código 9, que significa ignorado. Vamos assumir que o não 
preenchido é o mesmo que o código 9.

In [6]:
"""
A coluna 'CONSULTAS' não tinha nenhum não preenchido.
"""
df_sinasc_sel['ESTCIVMAE'].fillna(9, inplace = True) 
df_sinasc_sel.isna().sum()

C:\Users\Natanael_Quintiliano\AppData\Local\Temp\ipykernel_16824\1054981930.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sinasc_sel['ESTCIVMAE'].fillna(9, inplace = True)


LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

### 6. Missing Values V <a name="missingV"></a>

<div style="text-align: right"
     
[Voltar ao índice](#questoes)

Substitua os valores faltantes da quantitativa (QTDFILVIVO) por zero.
     

In [7]:
df_sinasc_sel['QTDFILVIVO'].fillna(0, inplace = True) 
df_sinasc_sel.isna().sum()

C:\Users\Natanael_Quintiliano\AppData\Local\Temp\ipykernel_16824\3566915320.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sinasc_sel['QTDFILVIVO'].fillna(0, inplace = True)


LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

### 7. Missing Values VI <a name="missingVI"></a>

<div style="text-align: right"
     
[Voltar ao índice](#questoes)
     
Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item
anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões
a todo o momento - não há necessariamente certo e errado aqui.

In [8]:
df_sinasc_sel['ESCMAE'].fillna('Ignorado', inplace = True) 
df_sinasc_sel['GESTACAO'].fillna('Ignorado', inplace = True) 
df_sinasc_sel['GRAVIDEZ'].fillna('Ignorado', inplace = True) 
df_sinasc_sel.isna().sum()

C:\Users\Natanael_Quintiliano\AppData\Local\Temp\ipykernel_16824\157349392.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sinasc_sel['ESCMAE'].fillna('Ignorado', inplace = True)
C:\Users\Natanael_Quintiliano\AppData\Local\Temp\ipykernel_16824\157349392.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sinasc_sel['GESTACAO'].fillna('Ignorado', inplace = True)
C:\Users\Natanael_Quintiliano\AppData\Local\Temp\ipykernel_16824\157349392.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64

### 8. Categorização  <a name="categorizacao"></a>

<div style="text-align: right"
     
[Voltar ao índice](#questoes)

Apgar possui uma classificação indicando se o bebê passou por asfixia:

 Entre 8 e 10 está em uma faixa 'normal'.     
 Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.     
 Entre 4 e 5 significa 'asfixia moderada'.     
 Entre 0 e 3 significa 'asfixia severa'.     
     
Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.

In [9]:
df_sinasc_sel.loc[df_sinasc_sel['APGAR5'] <= 3, 'CLASSIFICACAOAPGAR'] = 'Asfixia severa'
df_sinasc_sel.loc[(df_sinasc_sel['APGAR5'] >= 4) & 
                  (df_sinasc_sel['APGAR5'] <= 5), 'CLASSIFICACAOAPGAR'] = 'Asfixia moderada'
df_sinasc_sel.loc[(df_sinasc_sel['APGAR5'] >= 6) & 
                  (df_sinasc_sel['APGAR5'] <= 7), 'CLASSIFICACAOAPGAR'] = 'Asfixia leve'
df_sinasc_sel.loc[(df_sinasc_sel['APGAR5'] >= 8) & 
                  (df_sinasc_sel['APGAR5'] <= 10), 'CLASSIFICACAOAPGAR'] = 'Normal'

df_sinasc_sel.sample(15)

C:\Users\Natanael_Quintiliano\AppData\Local\Temp\ipykernel_16824\3469450027.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sinasc_sel.loc[df_sinasc_sel['APGAR5'] <= 3, 'CLASSIFICACAOAPGAR'] = 'Asfixia severa'


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,CLASSIFICACAOAPGAR
4940,1,20,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,Normal
1525,1,36,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,3,9.0,Normal
5832,1,31,2.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,Normal
26634,1,29,2.0,4 a 7 anos,1.0,37 a 41 semanas,Única,4,8.0,Normal
18986,1,38,1.0,8 a 11 anos,3.0,37 a 41 semanas,Única,3,10.0,Normal
4451,1,24,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,Normal
20400,1,20,2.0,4 a 7 anos,0.0,37 a 41 semanas,Única,4,9.0,Normal
17771,1,23,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,Normal
7900,1,27,1.0,8 a 11 anos,3.0,37 a 41 semanas,Única,1,9.0,Normal
21688,1,22,1.0,12 anos ou mais,0.0,32 a 36 semanas,Única,4,9.0,Normal


#### Calculando as frequências dessa categorização.

In [10]:
df_sinasc_sel['CLASSIFICACAOAPGAR'].value_counts(normalize = True)*100

CLASSIFICACAOAPGAR
Normal              98.284123
Asfixia leve         1.188487
Asfixia severa       0.274838
Asfixia moderada     0.252553
Name: proportion, dtype: float64

### 9. Renomear variaveis <a name="renomeando"></a>

<div style="text-align: right"
     
[Voltar ao índice](#questoes)
     
Renomeie as variáveis para que fiquem no snake case, ou seja, em letras minúsculas, com um underscore entre as palávras. 
Dica: repare que se você não quiser criar um dataframe novo, você vai precisar usar a opção inplace = True.

In [11]:
df_sinasc_sel['CLASSIFICACAOAPGAR'].replace(['Asfixia severa','Asfixia moderada','Asfixia leve','Normal'], 
                                             ['asfixia_severa','asfixia_moderada','asfixia_leve','normal'], inplace=True)

df_sinasc_sel.sample(10)

C:\Users\Natanael_Quintiliano\AppData\Local\Temp\ipykernel_16824\2593436702.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sinasc_sel['CLASSIFICACAOAPGAR'].replace(['Asfixia severa','Asfixia moderada','Asfixia leve','Normal'],


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,CLASSIFICACAOAPGAR
5852,1,27,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,8.0,normal
6526,1,37,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,3,10.0,normal
7211,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,2,10.0,normal
22743,1,21,5.0,4 a 7 anos,1.0,37 a 41 semanas,Única,3,9.0,normal
23726,1,26,2.0,12 anos ou mais,0.0,37 a 41 semanas,Única,4,9.0,normal
12985,1,28,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
4840,1,15,5.0,4 a 7 anos,0.0,42 semanas e mais,Única,4,8.0,normal
11269,1,32,2.0,12 anos ou mais,2.0,37 a 41 semanas,Única,4,9.0,normal
19513,1,20,1.0,12 anos ou mais,0.0,37 a 41 semanas,Única,4,9.0,normal
19311,1,33,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,4,9.0,normal
